# Model Comparison Summary
**Auto-extracted from your Colab notebooks**

In [ ]:
import json
import re
import pandas as pd
from pathlib import Path

# === CONFIG: Update these with your actual notebook names ===
NOTEBOOKS = {
    "MLP": "01_MLP.ipynb",
    "CNN-LSTM": "02_CNN_LSTM.ipynb",
    "Transformer": "03_Transformer.ipynb",
    "Transformer-Optimized": "04_Transformer_Optimized.ipynb"
}

# === Extract metrics from notebook outputs ===
results = {}

def extract_from_output(text):
    acc = re.search(r'accuracy.*?([0-9]\.[0-9]+)', text, re.I)
    f1 = re.search(r'f1.*?([0-9]\.[0-9]+)', text, re.I)
    params = re.search(r'params.*?([0-9]+ ?[KM]?)', text, re.I)
    return {
        'Accuracy': float(acc.group(1)) if acc else None,
        'F1': float(f1.group(1)) if f1 else None,
        'Params': params.group(1) if params else None
    }

for name, filename in NOTEBOOKS.items():
    path = Path('notebooks') / filename
    if not path.exists():
        results[name] = {'Accuracy': 'Missing', 'F1': 'Missing', 'Params': 'Missing'}
        continue
    
    with open(path, 'r', encoding='utf-8') as f:
        nb = json.load(f)
    
    output_text = ''
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            for output in cell.get('outputs', []):
                if 'text' in output:
                    output_text += ' '.join(output['text'])
                if 'data' in output and 'text/plain' in output['data']:
                    output_text += ' '.join(output['data']['text/plain'])
    
    metrics = extract_from_output(output_text)
    results[name] = metrics

# === Display ===
df = pd.DataFrame(results).T
df

In [ ]:
# Export to Markdown for README
print(df.to_markdown())